In [ ]:
import sys
import os

import numpy as np
import pandas as pd
#import holoviews as hv
#import networkx as nx
#import bokeh
#from holoviews import opts
import matplotlib.pyplot as plt

#hv.extension('bokeh')

sys.path.append('../../')
os.chdir("../../")

import configparser
import logging
import pathlib
import networkx as nx

from src.features.dataloader import DataLoader
from src.models.networkx_graph import SurfaceModel
from configuration.basic_configuration import configuration

load_test_data = configuration['PARAMETERS']['dataset'] == 'test'

if not load_test_data:
    base_path = configuration['PATHS']['model_data_dir']
else:
    base_path = configuration['PATHS']['test_data_dir']

logging.debug(f"base_path: {base_path}")

encoding = "iso-8859-1"

In [ ]:
logging.basicConfig(format='%(asctime)s - %(levelname)s: %(message)s', level=logging.INFO,
                    datefmt='%d.%m.%Y %H:%M:%S')
#####################################

#####################################
# Initiate data loader
# logging.info("Initiating data_loader")

# --> Load all data:
# loader = DataLoader(hdfs_pipe=False)  # hdfs_pipe = False --> files will be loaded directly from CSV
# patient_data = loader.patient_data()
#####################################

In [ ]:
movements_df = pd.read_csv(os.path.join(base_path, "../../interim/LA_ISH_NBEW.csv"), encoding=encoding, parse_dates=["Begin Datetime", "End Datetime"], index_col="Unnamed: 0")

# movements_df["Begin Time"] = movements_df["Begin Time"].str[:8]
# movements_df["End Time"] = movements_df["End Time"].str[:8]
#
# #movements_df["Begin Date"] = pd.to_datetime(movements_df["Begin Date"], format="%Y-%m-%d", errors="ignore")
# #movements_df["End Date"] = pd.to_datetime(movements_df["End Date"], format="%Y-%m-%d", errors="ignore")
#
# movements_df["Begin Datetime"] = (movements_df["Begin Date"] + " " + movements_df["Begin Time"])
# movements_df["End Datetime"] = (movements_df["End Date"] + " " + movements_df["End Time"])
# movements_df = movements_df.drop(labels=["Begin Date", "Begin Time", "End Date", "End Time"], axis=1)
#
# movements_df["Begin Datetime"] = pd.to_datetime(movements_df["Begin Datetime"], format="%Y-%m-%d %H:%M:%S", errors='coerce')
# movements_df["End Datetime"] = pd.to_datetime(movements_df["End Datetime"], format="%Y-%m-%d %H:%M:%S", errors='coerce')

#movements_df = movements_df.reset_index()
len(movements_df)

In [ ]:
movements_df["Begin Datetime"].apply(lambda x: type(x)).unique()
movements_df["End Datetime"].apply(lambda x: type(x)).unique()

#pd.to_datetime(pd.DataFrame(data={"d":["1677-12-31"]})["d"], format="%Y-%m-%d")
#movements_df.sort_values("End Datetime")

In [ ]:
movements_df

In [ ]:
movements_df["Begin Datetime"].groupby([movements_df["Begin Datetime"].dt.year, movements_df["Begin Datetime"].dt.month]).count().plot(kind="bar", figsize=(20,16))

In [ ]:
case_df = pd.read_csv(os.path.join(base_path, "../../interim/DIM_FALL.csv"), encoding=encoding, index_col="Unnamed: 0")
case_df = case_df.set_index("Case ID")
case_df

In [ ]:
movements_with_cases_df = movements_df.join(case_df[["Case Status", "Patient ID"]], on="Case ID", how="left")

#movements_with_cases_df[movements_with_cases_df["Case Status"].apply(lambda x: not isinstance(x, str))]

In [ ]:
patient_df = pd.read_csv(os.path.join(base_path, "../../interim/DIM_PATIENT.csv"), encoding=encoding, index_col="Unnamed: 0", dtype=str)
patient_df = patient_df.set_index("Patient ID")
patient_df
#patient_df[patient_df["Gender"].apply(lambda x: not isinstance(x, str))]

In [ ]:
movements_with_patients_df = movements_with_cases_df[movements_with_cases_df["Patient ID"].isin(patient_df.index)]#.join(patient_df["Gender"], on="Patient ID", how="left")

In [ ]:
movements_with_patients_df

In [ ]:
vre_screening_data = pd.read_csv(os.path.join(base_path, "../../interim/VRE_SCREENING_DATA.csv"), encoding=encoding, index_col="Unnamed: 0", dtype=str)
#vre_screening_data['Patient ID'] = pd.to_numeric(vre_screening_data["Patient ID"], errors="coerce")
vre_screening_data

In [ ]:
vre_screening_with_patient = vre_screening_data[vre_screening_data["Patient ID"].isin(patient_df.index)]
vre_screening_with_patient

In [ ]:
vre_screening_with_case = vre_screening_data[vre_screening_data["Patient ID"].isin(set(case_df["Patient ID"].to_list()))]
vre_screening_with_case